In [ ]:
!pip install paddlepaddle paddleocr

In [ ]:
import os
import cv2
import time
import requests
from io import BytesIO
from PIL import Image
from joblib import Parallel, delayed
from paddleocr import PaddleOCR
import numpy as np
import pandas as pd

# Define the dataset path (CSV file)
dataset_path = '/content/train.csv'

# Define the output CSV file path with extracted text
output_csv = '/content/train_with_text.csv'

# Define counters for successful extractions and errors
success_count = 0
error_count = 0

# Define the current time
start_time = time.time()

# Function to upscale the image
def upscale_image(image, scale_factor=2):
    """
    Upscales an image by the given scale factor.
    """
    width = int(image.shape[1] * scale_factor)
    height = int(image.shape[0] * scale_factor)
    dimensions = (width, height)

    # Resize the image to the new dimensions
    upscaled_image = cv2.resize(image, dimensions, interpolation=cv2.INTER_CUBIC)
    return upscaled_image

# Function to download image from URL and convert it to an OpenCV image
def download_image_from_url(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for failed requests
        image = Image.open(BytesIO(response.content)).convert('RGB')  # Convert to RGB
        return np.array(image)  # Convert the PIL image to a NumPy array (OpenCV format)
    except Exception as e:
        print(f"Error downloading image {url}: {str(e)}")
        return None

# Define a function for text extraction using PaddleOCR
def extract_text(image_path, scale_factor=2):
    try:
        # Initialize PaddleOCR within the function to avoid PicklingError
        ocr = PaddleOCR(use_angle_cls=True, lang='en')

        # Check if image_path is a URL (starts with 'http' or 'https')
        if image_path.startswith('http'):
            # Download the image from the URL
            image_cv = download_image_from_url(image_path)
        else:
            # Load the image from a local file
            image_cv = cv2.imread(image_path)

        if image_cv is None:
            raise ValueError("Image could not be loaded, possibly corrupted or invalid format.")

        # Upscale the image to enhance text extraction
        image_upscaled = upscale_image(image_cv, scale_factor=scale_factor)

        # Convert the image from BGR to RGB (PaddleOCR expects RGB format)
        image_rgb = cv2.cvtColor(image_upscaled, cv2.COLOR_BGR2RGB)

        # Extract text using PaddleOCR
        results = ocr.ocr(image_rgb)  # Returns a list of text boxes and text

        # Join the results into a single string
        text = '\n'.join([res[1][0] for res in results[0]])  # Extract text part

        return text

    except Exception as e:
        # Handle exceptions (e.g., if the image is corrupted)
        print(f"Error processing {image_path}: {str(e)}")
        return None

# Read the dataset into a pandas DataFrame
df = pd.read_csv(dataset_path)

# Limit the DataFrame to the first 50 rows
df = df.head(30)


# Ensure that there is an 'image_link' column in the dataset
if 'image_link' not in df.columns:
    raise ValueError("The dataset must contain an 'image_link' column with image paths or URLs.")

# Initialize a new column for extracted text
df['extracted_text'] = None

# Iterate over the dataset and extract text for each image
for idx, row in df.iterrows():
    image_path = row['image_link']
    extracted_text = extract_text(image_path)  # Extract text from the image

    if extracted_text is not None:
        df.at[idx, 'extracted_text'] = extracted_text
        success_count += 1
    else:
        error_count += 1

# Save the updated dataset with the extracted text to a new CSV file
df.to_csv(output_csv, index=False)

# Calculate the total execution time in minutes
end_time = time.time()
total_time_seconds = end_time - start_time
total_time_minutes = total_time_seconds / 60  # Convert to minutes

# Print the total time taken in minutes
print(f"Total time taken: {total_time_minutes:.2f} minutes")

# Print the count of successful extractions and errors
print(f"Images successfully extracted: {success_count}")
print(f"Images with errors: {error_count}")

# Print the path to the output CSV file
print(f"Updated dataset saved to: {output_csv}")

In [ ]:
test_ocr = pd.read_csv("/content/test_ocr_results.csv")

test_ocr["image_link"] = test_ocr["Image Path"]
test_ocr["extracted_text"] = test_ocr["Extracted Text"]

test_ocr = test_ocr.drop(columns=['Processing Time (seconds)', 'Image Path', 'Extracted Text'])
test_ocr

In [ ]:
test = pd.read_csv("/content/test.csv")

# Define the substring to remove
substring_to_remove = 'https://m.media-amazon.com/images/I/'

# Remove the substring from the 'links' column
test['image_link'] = test['image_link'].str.replace(substring_to_remove, '', regex=False)

In [ ]:
# Merge the two dataframes on the 'image_link' column (left join to preserve all test_df rows)
merged_df = pd.merge(test, test_ocr, on='image_link', how='left')

In [ ]:
df = merged_df

In [ ]:
import pandas as pd
df = pd.read_csv("/content/test_merged.csv")

In [ ]:


import re

# Function to build the pattern for a list of units
def build_pattern(units):
    # Sort units by length descending to match longer units first
    units_sorted = sorted(units, key=lambda x: -len(x))
    # Escape units that have special regex characters
    units_escaped = [re.escape(u) for u in units_sorted]
    # Build the unit pattern, adding word boundaries
    units_pattern = '|'.join(units_escaped)
    # Build the final pattern
    pattern = r"(\d+\.?\d*)\s?(" + units_pattern + r")\b"
    return pattern

# Define units per entity
weight_units = ['milligram', 'mg', 'miligram', 'milligrams', 'miligrammes',
                'gram', 'g', 'grams',
                'kilogram', 'kg',
                'microgram', 'µg',
                'ounce', 'oz',
                'pound', 'lb',
                'ton']

length_units = ['millimetre', 'mm', 'millimeter',
                'centimetre', 'cm', 'centimeter',
                'metre', 'meter', r'm(?!m)',  # Negative lookahead to prevent matching 'mm' as 'm'
                'foot', 'ft',
                'inch', 'in',
                'yard', 'yd']

voltage_units = ['kilovolt', 'kV', 'kv',
                 'millivolt', 'mV', 'mv',
                 'volt', 'V', 'v']

wattage_units = ['kilowatt', 'kW', 'kw', 'watt', 'W', 'w']

volume_units = ['centilitre', 'cl',
                'cubic foot', 'ft³',
                'cubic inch', 'in³',
                'cup',
                'decilitre', 'dl',
                'fluid ounce', 'fl oz',
                'gallon', 'imperial gallon',
                'litre', 'liter',
                'millilitre', 'ml', 'milliliter',
                'microlitre', 'microliter',
                'pint', 'quart']

# Define the extraction patterns for each entity type
patterns = {
    'item_weight': build_pattern(weight_units),
    'depth': build_pattern(length_units),
    'width': build_pattern(length_units),
    'height': build_pattern(length_units),
    'voltage': build_pattern(voltage_units),
    'wattage': build_pattern(wattage_units),
    'item_volume': build_pattern(volume_units),
    'maximum_weight_recommendation': build_pattern(weight_units)
}

# Normalize unit names (ensure all keys are in lowercase)
unit_mappings = {
    # weight units
    'mg': 'milligram', 'milligram': 'milligram', 'miligram': 'milligram',
    'milligrams': 'milligram', 'miligrammes': 'milligram',
    'g': 'gram', 'gram': 'gram', 'grams': 'gram',
    'kg': 'kilogram', 'kilogram': 'kilogram',
    'µg': 'microgram', 'microgram': 'microgram',
    'oz': 'ounce', 'ounce': 'ounce',
    'lb': 'pound', 'pound': 'pound',
    'ton': 'ton',

    # length units
    'cm': 'centimetre', 'centimetre': 'centimetre', 'centimeter': 'centimetre',
    'mm': 'millimetre', 'millimetre': 'millimetre', 'millimeter': 'millimetre',
    'm': 'metre', 'metre': 'metre', 'meter': 'metre',
    'foot': 'foot', 'ft': 'foot', 'inch': 'inch', 'in': 'inch',
    'yard': 'yard', 'yd': 'yard',

    # voltage units
    'kv': 'kilovolt', 'kilovolt': 'kilovolt',
    'mv': 'millivolt', 'millivolt': 'millivolt',
    'v': 'volt', 'volt': 'volt',

    # wattage units
    'kw': 'kilowatt', 'kilowatt': 'kilowatt',
    'w': 'watt', 'watt': 'watt',

    # volume units
    'cl': 'centilitre', 'centilitre': 'centilitre',
    'dl': 'decilitre', 'decilitre': 'decilitre',
    'ft³': 'cubic foot', 'cubic foot': 'cubic foot',
    'in³': 'cubic inch', 'cubic inch': 'cubic inch',
    'cup': 'cup',
    'fluid ounce': 'fluid ounce', 'fl oz': 'fluid ounce',
    'gallon': 'gallon', 'imperial gallon': 'imperial gallon',
    'litre': 'litre', 'liter': 'litre',
    'ml': 'millilitre', 'millilitre': 'millilitre', 'milliliter': 'millilitre',
    'microlitre': 'microlitre', 'microliter': 'microlitre',
    'pint': 'pint', 'quart': 'quart'
}

# Function to extract and normalize quantity and unit from text
def extract_quantity(row):
    text = row['extracted_text']
    entity_name = row['entity_name']

    # Skip if text is None
    if pd.isna(text):
        return ''

    pattern = patterns.get(entity_name)
    if pattern:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            value, unit = match.groups()
            value = float(value)
            normalized_unit = unit_mappings.get(unit.lower(), unit.lower())
            return f"{value} {normalized_unit}"

    return ''

# # Function to check if extracted text matches the entity value
# def check_match(row):
#     extracted = row['extracted_text']
#     entity_value = row['entity_value']

#     return extracted == entity_value

# Apply the function to the DataFrame
df['extracted_text'] = df.apply(extract_quantity, axis=1)
# df['match'] = df.apply(check_match, axis=1)

In [ ]:
import pandas as pd
import re

# Function to build the pattern for a list of units
def build_pattern(units):
    # Sort units by length descending to match longer units first
    units_sorted = sorted(units, key=lambda x: -len(x))
    # Escape units that have special regex characters
    units_escaped = [re.escape(u) for u in units_sorted]
    # Build the unit pattern, adding word boundaries
    units_pattern = '|'.join(units_escaped)
    # Build the final pattern
    pattern = r"(\d+\.?\d*)\s?(" + units_pattern + r")\b"
    return pattern

# Define units per entity
weight_units = ['milligram', 'mg', 'miligram', 'milligrams', 'miligrammes',
                'gram', 'g', 'grams',
                'kilogram', 'kg',
                'microgram', 'µg',
                'ounce', 'oz',
                'pound', 'lb',
                'ton']

length_units = ['millimetre', 'mm', 'millimeter',
                'centimetre', 'cm', 'centimeter',
                'metre', 'meter', r'm(?!m)',  # Negative lookahead to prevent matching 'mm' as 'm'
                'foot', 'ft',
                'inch', 'in',
                'yard', 'yd']

voltage_units = ['kilovolt', 'kV', 'kv',
                 'millivolt', 'mV', 'mv',
                 'volt', 'V', 'v']

wattage_units = ['kilowatt', 'kW', 'kw', 'watt', 'W', 'w']

volume_units = ['centilitre', 'cl',
                'cubic foot', 'ft³',
                'cubic inch', 'in³',
                'cup',
                'decilitre', 'dl',
                'fluid ounce', 'fl oz',
                'gallon', 'imperial gallon',
                'litre', 'liter',
                'millilitre', 'ml', 'milliliter',
                'microlitre', 'microliter',
                'pint', 'quart']

# Define the extraction patterns for each entity type
patterns = {
    'item_weight': build_pattern(weight_units),
    'depth': build_pattern(length_units),
    'width': build_pattern(length_units),
    'height': build_pattern(length_units),
    'voltage': build_pattern(voltage_units),
    'wattage': build_pattern(wattage_units),
    'item_volume': build_pattern(volume_units),
    'maximum_weight_recommendation': build_pattern(weight_units)
}

# Normalize unit names (ensure all keys are in lowercase)
unit_mappings = {
    # weight units
    'mg': 'milligram', 'milligram': 'milligram', 'miligram': 'milligram',
    'milligrams': 'milligram', 'miligrammes': 'milligram',
    'g': 'gram', 'gram': 'gram', 'grams': 'gram',
    'kg': 'kilogram', 'kilogram': 'kilogram',
    'µg': 'microgram', 'microgram': 'microgram',
    'oz': 'ounce', 'ounce': 'ounce',
    'lb': 'pound', 'pound': 'pound',
    'ton': 'ton',

    # length units
    'cm': 'centimetre', 'centimetre': 'centimetre', 'centimeter': 'centimetre',
    'mm': 'millimetre', 'millimetre': 'millimetre', 'millimeter': 'millimetre',
    'm': 'metre', 'metre': 'metre', 'meter': 'metre',
    'foot': 'foot', 'ft': 'foot', 'inch': 'inch', 'in': 'inch',
    'yard': 'yard', 'yd': 'yard',

    # voltage units
    'kv': 'kilovolt', 'kilovolt': 'kilovolt',
    'mv': 'millivolt', 'millivolt': 'millivolt',
    'v': 'volt', 'volt': 'volt',

    # wattage units
    'kw': 'kilowatt', 'kilowatt': 'kilowatt',
    'w': 'watt', 'watt': 'watt',

    # volume units
    'cl': 'centilitre', 'centilitre': 'centilitre',
    'dl': 'decilitre', 'decilitre': 'decilitre',
    'ft³': 'cubic foot', 'cubic foot': 'cubic foot',
    'in³': 'cubic inch', 'cubic inch': 'cubic inch',
    'cup': 'cup',
    'fluid ounce': 'fluid ounce', 'fl oz': 'fluid ounce',
    'gallon': 'gallon', 'imperial gallon': 'imperial gallon',
    'litre': 'litre', 'liter': 'litre',
    'ml': 'millilitre', 'millilitre': 'millilitre', 'milliliter': 'millilitre',
    'microlitre': 'microlitre', 'microliter': 'microlitre',
    'pint': 'pint', 'quart': 'quart'
}

# Conversion factors to millimetres for length units
unit_to_mm = {
    'millimetre': 1.0, 'mm': 1.0, 'millimeter': 1.0,
    'centimetre': 10.0, 'cm': 10.0, 'centimeter': 10.0,
    'metre': 1000.0, 'm': 1000.0, 'meter': 1000.0,
    'foot': 304.8, 'ft': 304.8,
    'inch': 25.4, 'in': 25.4,
    'yard': 914.4, 'yd': 914.4
}

# Function to extract and normalize quantity and unit from text
def extract_quantity(row):
    text = row['extracted_text']
    entity_name = row['entity_name']

    # Skip if text is None
    if pd.isna(text):
        return ''

    pattern = patterns.get(entity_name)
    if pattern:
        # For height/depth/width/breadth, extract all matches
        if entity_name in ['height', 'depth', 'width', 'breadth']:
            matches = re.findall(pattern, text, re.IGNORECASE)
            if matches:
                # Convert matches to list of (value_in_mm, value_original, unit)
                values_units = []
                for match in matches:
                    value_str, unit = match
                    value = float(value_str)
                    normalized_unit = unit_mappings.get(unit.lower(), unit.lower())
                    conversion_factor = unit_to_mm.get(normalized_unit.lower())
                    if conversion_factor is None:
                        # Skip if unit is not recognized
                        continue
                    value_in_mm = value * conversion_factor
                    values_units.append((value_in_mm, value, normalized_unit))
                if not values_units:
                    return ''
                # Decide whether to pick the largest or smallest value
                if entity_name in ['height', 'depth']:
                    # Pick the largest value
                    selected = max(values_units, key=lambda x: x[0])
                elif entity_name in ['width', 'breadth']:
                    # Pick the smallest value
                    selected = min(values_units, key=lambda x: x[0])
                _, selected_value, selected_unit = selected
                return f"{selected_value} {selected_unit}"
        else:
            match = re.search(pattern, text, re.IGNORECASE)
            if match:
                value, unit = match.groups()
                value = float(value)
                normalized_unit = unit_mappings.get(unit.lower(), unit.lower())
                return f"{value} {normalized_unit}"

    return ''

# Apply the function to the DataFrame
df['extracted_text'] = df.apply(extract_quantity, axis=1)


In [ ]:
df.shape

(131187, 6)

In [ ]:
from sklearn.metrics import f1_score
y_true = df['entity_value'].astype(str)
y_pred = df['extracted_text'].astype(str)
f1 = f1_score(y_true, y_pred, average='weighted')
print(f"F1 Score for training data: {f1:.4f}")

F1 Score for training data: 0.4064


In [ ]:
df

,Unnamed: 0,index,image_link,group_id,entity_name,extracted_text
0,0,0,110EibNyclL.jpg,156839,height,199.39 centimetre
1,1,1,11TU2clswzL.jpg,792578,width,42.0 centimetre
2,2,2,11TU2clswzL.jpg,792578,height,200.0 centimetre
3,3,3,11TU2clswzL.jpg,792578,depth,200.0 centimetre
4,4,4,11gHj8dhhrL.jpg,792578,depth,90.0 centimetre
...,...,...,...,...,...,...
131182,131182,131283,A1rVsIzEtkL.jpg,721522,maximum_weight_recommendation,500.0 pound
131183,131183,131284,A1rdvZ5zDdL.jpg,603688,item_weight,
131184,131184,131285,A1rdvZ5zDdL.jpg,603688,maximum_weight_recommendation,
131185,131185,131286,A1tnTUPyr7L.jpg,853009,item_weight,


In [ ]:
df = df.drop(columns=["image_link", "entity_name"])

In [ ]:
y_pred = df['extracted_text']
index = df['index']


submission = pd.DataFrame({
    'index': index, # Using index as ID
    'prediction': y_pred
})

submission.to_csv('submission_2.csv', index=False)

NameError: name 'df' is not defined

In [ ]:
import pandas as pd
df = pd.read_csv("/content/submission_1.csv")

In [ ]:
import pandas as pd
import re
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import KNNImputer

# Step 1: Extract numerical value and unit, and standardize units
def extract_value_and_unit(prediction):
    if not isinstance(prediction, str):  # Check if the input is a string
        return None, None
    match = re.match(r"([0-9.]+)\s*(\w+)", prediction)
    if match:
        value = float(match.group(1))
        unit = match.group(2).lower()
        # Standardize units to a common scale
        if unit in ['gram', 'grams']:
            value = value / 1000  # Convert grams to kilograms
        elif unit in ['centimetre', 'centimetres']:
            value = value / 100  # Convert centimetres to meters
        return value, unit
    return None, None

# Apply the extraction to the 'prediction' column
df['numeric_prediction'], df['unit'] = zip(*df['prediction'].apply(lambda x: extract_value_and_unit(x)))

# Step 2: Encode group_id using LabelEncoder
le = LabelEncoder()
df['group_id_encoded'] = le.fit_transform(df['group_id'])

# Step 3: Prepare data for KNN imputation
# Select the relevant columns for KNN
X = df[['group_id_encoded', 'numeric_prediction']]

# Step 4: Apply KNN to fill missing values
# Initialize KNN Imputer (k=5)
imputer = KNNImputer(n_neighbors=2)

# Fit and transform the dataset (this will fill in missing values)
X_imputed = imputer.fit_transform(X)

# Put the imputed values back into the dataframe
df['numeric_prediction_imputed'] = X_imputed[:, 1]

# Step 5: Convert back to original units
def convert_to_original_units(value, original_unit):
    if original_unit in ['gram', 'grams']:
        value = value * 1000  # Convert kilograms back to grams
    elif original_unit in ['centimetre', 'centimetres']:
        value = value * 100  # Convert meters back to centimetres
    return f"{value:.2f} {original_unit}"

# Apply the conversion to the imputed values
df['prediction_imputed'] = df.apply(
    lambda row: convert_to_original_units(row['numeric_prediction_imputed'], row['unit'])
    if pd.isna(row['prediction']) else row['prediction'], axis=1)

# Display the final DataFrame
print(df[['group_id', 'prediction', 'numeric_prediction', 'prediction_imputed']])


        group_id       prediction  numeric_prediction   prediction_imputed
0         156839        2.63 inch               2.630            2.63 inch
1         792578  42.0 centimetre               0.420      42.0 centimetre
2         792578  42.0 centimetre               0.420      42.0 centimetre
3         792578  42.0 centimetre               0.420      42.0 centimetre
4         792578  10.5 centimetre               0.105      10.5 centimetre
...          ...              ...                 ...                  ...
131182    721522      500.0 pound             500.000          500.0 pound
131183    603688              NaN                 NaN  15522352378.85 None
131184    603688              NaN                 NaN  15522352378.85 None
131185    853009              NaN                 NaN            1.40 None
131186    853009              NaN                 NaN            1.40 None

[131187 rows x 4 columns]


In [ ]:
df['prediction_imputed'].iloc[100:105]

,prediction_imputed
100,37.0 inch
101,59.0 millilitre
102,15522352405.00 None
103,90.0 centimetre
104,40.0 centimetre
